# Calculation training けいさんれんしゅう

In [28]:
import random
from IPython.display import display, Math, Latex, clear_output
import ipywidgets as widgets

# 問題生成関数
def generate_math_problem():
    operation = random.choice(['addition', 'subtraction', 'multiplication', 'division'])
    num1, num2, answer, hint = None, None, None, None
    
    if operation == 'addition':
        # 足し算：2桁+2桁、答えが1000以下
        num1 = random.randint(10, 99)
        num2 = random.randint(10, 99)
        answer = num1 + num2
        if answer > 1000:
            return generate_math_problem()  # 1000以上なら再生成
        hint = f"{num1 // 10}□ + {num2 // 10} = {answer // 10}□"
    
    elif operation == 'subtraction':
        # 引き算：2桁-2桁、答えがマイナスにならない
        num1 = random.randint(10, 99)
        num2 = random.randint(10, num1)  # num1が常に大きい
        answer = num1 - num2
        hint = f"{num1 // 10}□ - {num2 // 10} = {answer // 10}□"
    
    elif operation == 'multiplication':
        # 掛け算：1桁×2桁、答えが1000以下
        num1 = random.randint(1, 9)
        num2 = random.randint(10, 99)
        answer = num1 * num2
        if answer > 1000:
            return generate_math_problem()  # 1000以上なら再生成
        hint = f"{num1} × {num2 // 10}□ = {answer // 10}□"
    
    elif operation == 'division':
        # 割り算：2桁÷1桁、あまりが出ない
        num2 = random.randint(1, 9)
        answer = random.randint(10, 99)
        num1 = num2 * answer  # num1はnum2と割り切れる数
        hint = f"{num1 // 10}□ ÷ {num2} = {answer // 10}□"
    
    return (operation, num1, num2, answer, hint)

# 問題のリストを生成
problems = [generate_math_problem() for _ in range(5)]


In [29]:
from IPython.display import display, Math, Latex, clear_output
import ipywidgets as widgets

# 問題とヒントを表示し、解答をチェックする関数
def display_problem_with_hint(problems):
    correct_count = [0]  # リストで包むことで、クロージャ内でのスコープ管理を行う
    output_all = widgets.Output()  # 最終的な成績を表示するエリア
    
    for i, (operation, num1, num2, answer, hint) in enumerate(problems):
        # 問題の表示
        display(Math(f"問題 {i+1}: {num1} {'+' if operation == 'addition' else '-' if operation == 'subtraction' else '×' if operation == 'multiplication' else '÷'} {num2} = ?"))

        # 出力エリアを作成
        output = widgets.Output()
        
        # ヒントを表示するためのボタンを作成
        hint_button = widgets.Button(description="ヒントを見る", button_style='info')
        
        # 解答入力テキストボックス
        answer_input = widgets.Text(placeholder='答えを入力してください')
        
        # 解答確認ボタン
        check_button = widgets.Button(description="解答を確認する", button_style='success')
        
        # 答えの正誤を表示するためのエリア
        result_output = widgets.Output()

        # ボタンクリック時のイベントハンドラ - ヒント表示
        def on_hint_button_clicked(b, output=output, operation=operation, num1=num1, num2=num2):
            with output:
                output.clear_output()  # 以前の出力をクリア
                
                # 問題の内容に応じたヒントを生成
                if operation == 'addition':
                    hint_text = generate_addition_hint(num1, num2)
                    display(Latex(hint_text))
                elif operation == 'subtraction':
                    hint_text = generate_subtraction_hint(num1, num2)
                    display(Latex(hint_text))
                elif operation == 'multiplication':
                    hint_text = generate_multiplication_hint(num1, num2)
                    display(Latex(hint_text))
                elif operation == 'division':
                    hint_text = generate_division_hint(num1, num2)
                    display(Latex(hint_text))
        
        # 解答を確認するボタンのイベントハンドラ - 正誤チェック
        def on_check_button_clicked(b, user_answer=answer_input, correct_answer=answer, result_output=result_output):
            with result_output:
                result_output.clear_output()  # 以前の出力をクリア
                try:
                    if int(user_answer.value) == correct_answer:
                        display(Latex(r"\text{正解！}"))
                        correct_count[0] += 1  # リスト内の正解数をカウント
                    else:
                        display(Latex(r"\text{不正解です。正解は " + str(correct_answer) + r" です。}"))  # 不正解時の表示を追加
                except ValueError:
                    display(Latex(r"\text{数字で答えを入力してください。}"))
        
        # ボタンにクリックイベントハンドラを設定
        hint_button.on_click(on_hint_button_clicked)
        check_button.on_click(on_check_button_clicked)
        
        # ボタンと出力エリアを表示
        display(answer_input)
        display(check_button, result_output)
        display(hint_button, output)
    
    # 成績を表示するボタン
    show_score_button = widgets.Button(description="成績を表示", button_style='primary')

    # 成績表示ボタンのイベントハンドラ
    def on_show_score_button_clicked(b):
        with output_all:
            output_all.clear_output()  # 以前の出力をクリア
            score = (correct_count[0] / len(problems)) * 100
            display(Latex(r"\text{あなたの成績: }" + str(correct_count[0]) + r"/" + str(len(problems)) + r"\text{ 正解}"))
            display(Latex(r"\text{スコア: }" + str(score) + r"\%"))
    
    # 成績表示ボタンにイベントハンドラを設定
    show_score_button.on_click(on_show_score_button_clicked)
    
    # 最終的な成績表示ボタンを表示
    display(show_score_button, output_all)

# 各計算のヒント生成関数
def generate_addition_hint(num1, num2):
    # 足し算の筆算ヒントを生成
    ones_place_sum = (num1 % 10) + (num2 % 10)
    tens_place_sum = (num1 // 10) + (num2 // 10) + (ones_place_sum // 10)
    hint_text = f"1の位を計算します: {num1 % 10} + {num2 % 10} = {ones_place_sum}。次に10の位を計算します: {num1 // 10} + {num2 // 10} + 繰り上がり = {tens_place_sum}。"
    return hint_text

def generate_subtraction_hint(num1, num2):
    # 引き算の筆算ヒントを生成
    if (num1 % 10) < (num2 % 10):  # 繰り下がりが必要な場合
        ones_place_diff = (num1 % 10 + 10) - (num2 % 10)
        tens_place_diff = (num1 // 10 - 1) - (num2 // 10)
        hint_text = f"1の位が小さいため10を借ります: {num1 % 10 + 10} - {num2 % 10} = {ones_place_diff}。次に10の位を計算します: {num1 // 10 - 1} - {num2 // 10} = {tens_place_diff}。"
    else:
        ones_place_diff = (num1 % 10) - (num2 % 10)
        tens_place_diff = (num1 // 10) - (num2 // 10)
        hint_text = f"1の位を計算します: {num1 % 10} - {num2 % 10} = {ones_place_diff}。次に10の位を計算します: {num1 // 10} - {num2 // 10} = {tens_place_diff}。"
    return hint_text

def generate_multiplication_hint(num1, num2):
    # 掛け算の筆算ヒントを生成
    ones_place_mul = (num2 % 10) * num1
    tens_place_mul = (num2 // 10) * num1
    hint_text = f"1の位でかけます: {num1} × {num2 % 10} = {ones_place_mul}。次に10の位でかけます: {num1} × {num2 // 10} = {tens_place_mul}。最後に合計します: {ones_place_mul} + {tens_place_mul}。"
    return hint_text

def generate_division_hint(num1, num2):
    # 割り算の筆算ヒントを生成
    quotient = num1 // num2
    remainder = num1 % num2
    hint_text = f"商を求めます: {num1} ÷ {num2} = {quotient}、あまり {remainder}。"
    return hint_text

# セッション1で生成された問題を表示
display_problem_with_hint(problems)


<IPython.core.display.Math object>

Text(value='', placeholder='答えを入力してください')

Button(button_style='success', description='解答を確認する', style=ButtonStyle())

Output()

Button(button_style='info', description='ヒントを見る', style=ButtonStyle())

...

Output()

Button(button_style='info', description='ヒントを見る', style=ButtonStyle())

Output()

Button(button_style='primary', description='成績を表示', style=ButtonStyle())

Output()